# Simplified Sandwich Uncertainty Propagation Test

This notebook demonstrates the improved sandwich uncertainty propagation with:
- **Automatic energy grid validation**
- **Proper relative/absolute sensitivity conversion**  
- **Enhanced results display with correlation effects**
- **Simplified interface**

In [1]:
# Import and reload updated sandwich module
import importlib
from mcnpy.UQ import sandwich
importlib.reload(sandwich)
from mcnpy.UQ.sandwich import sandwich_uncertainty_propagation, filter_reactions_by_nuclide

In [2]:
# Reload the modules to get the updated function
import importlib
from mcnpy.serpent.parse_sens import load_sensitivity_file
from mcnpy.sensitivities import sensitivity_processing
importlib.reload(sensitivity_processing)
from mcnpy.sensitivities.sensitivity_processing import create_sdf_from_serpent
from mcnpy.cov.parse_covmat import read_scale_covmat, read_njoy_covmat

In [3]:
# Load files
sens_Fe56 = load_sensitivity_file("/home/MONLEON-JUAN/MCNPy/mcnpy/serpent/PWRSphere.sss2_sens0.m")
sens_H1 = load_sensitivity_file("/share_snc/snc/JuanMonleon/PhD_Year1_Workspace/SERPENT2.2/PWRSphere/full_composition/44-scale/WATER/SRx-1001/PWRSphere.sss2_sens0.m")
sens_Fe54 = load_sensitivity_file("/share_snc/snc/JuanMonleon/PhD_Year1_Workspace/SERPENT2.2/PWRSphere/full_composition/44-scale/STEEL/SRx-26054/PWRSphere.sss2_sens0.m")
sens_O16 = load_sensitivity_file("/share_snc/snc/JuanMonleon/PhD_Year1_Workspace/SERPENT2.2/PWRSphere/full_composition/44-scale/WATER/SRx-8016/PWRSphere.sss2_sens0.m")
scale_file_path = '/home/MONLEON-JUAN/MCNPy/mcnpy/cov/files/scale.rev05.44groupcov.txt'

In [4]:
# Create SDF data from SERPENT results
sdf_data = create_sdf_from_serpent(
    serpent_file=[sens_Fe56,sens_H1, sens_Fe54, sens_O16],
    response_name="sens_ratio_BIN_1",  # or whatever response you're interested in
    title="SERPENT Sensitivity Analysis",
    response_values=(6.18400E-04, 0.01544)  # optional: set reference response values
)
sdf_data.group_inelastic_reactions(replace=True)

Processing file 1/4 with response 'sens_ratio_BIN_1'...
Processing file 2/4 with response 'sens_ratio_BIN_1'...
Processing file 3/4 with response 'sens_ratio_BIN_1'...
Processing file 4/4 with response 'sens_ratio_BIN_1'...
Combined SDF contains 18 sensitivity profiles from 4 files
Created MT 4 for Fe-56 (ZAID 26056)
Removed MT 51, 52, 53 for Fe-56 (ZAID 26056)
Created MT 4 for Fe-54 (ZAID 26054)
Removed MT 51, 52, 53 for Fe-54 (ZAID 26054)


In [5]:
sdf_data.write_file("output_directory")

SDF file saved successfully: output_directory/SERPENT_Sensitivity_Analysis_sens_ratio_BIN_1.sdf


In [6]:
covmat = read_scale_covmat(scale_file_path)

In [7]:
# Simple test: Run uncertainty propagation with all available data
print("🚀 Running sandwich uncertainty propagation...")
print("   (All validation and energy matching done automatically)")

result = sandwich_uncertainty_propagation(
    sdf_data=sdf_data,
    cov_mat=covmat,
    verbose=False
)

print("\n" + "="*70)
print("RESULTS:")
print("="*70)
print(result)

🚀 Running sandwich uncertainty propagation...
   (All validation and energy matching done automatically)

RESULTS:
UNCERTAINTY PROPAGATION RESULTS (Sandwich Formula)
Total variance (σ²):          4.203924e-03
Total uncertainty (1σ):       6.483767e-02 (relative)
Relative uncertainty:          6.4838%

RESPONSE VALUE WITH UNCERTAINTY:
Response value:                6.184000e-04 ± 9.548096e-06 (statistical)
Nuclear data uncertainty:      ± 4.009562e-05 (absolute)
Nuclear data uncertainty:      ± 6.48% (relative)
Final result:                  6.184000e-04 ± 4.009562e-05

Reactions included:            10
Energy groups:                 44
Cross-correlation effects:     -6.607614e-04 (15.7% of total)

INDIVIDUAL REACTION CONTRIBUTIONS
Rank Nuclide      Reaction        Variance     % of Total
----------------------------------------------------------------------
1    Fe-56        (z,z0)          2.0918e-03    49.76%
2    H-1          (z,z0)          1.3904e-03    33.07%
3    Fe-56        (z

In [8]:
# Test with specific reaction filter (Fe-56 elastic only)
print("🎯 Testing with Fe-56 elastic scattering only...")

result_elastic = sandwich_uncertainty_propagation(
    sdf_data=sdf_data,
    cov_mat=covmat,
    reaction_filter=filter_reactions_by_nuclide(26056, [2]),  # Fe-56 elastic only
    verbose=False  # Less verbose for cleaner output
)

print(result_elastic)

🎯 Testing with Fe-56 elastic scattering only...
UNCERTAINTY PROPAGATION RESULTS (Sandwich Formula)
Total variance (σ²):          2.415997e-03
Total uncertainty (1σ):       4.915279e-02 (relative)
Relative uncertainty:          4.9153%

RESPONSE VALUE WITH UNCERTAINTY:
Response value:                6.184000e-04 ± 9.548096e-06 (statistical)
Nuclear data uncertainty:      ± 3.039609e-05 (absolute)
Nuclear data uncertainty:      ± 4.92% (relative)
Final result:                  6.184000e-04 ± 3.039609e-05

Reactions included:            1
Energy groups:                 44
Cross-correlation effects:     None (independent reactions)

INDIVIDUAL REACTION CONTRIBUTIONS
Rank Nuclide      Reaction        Variance     % of Total
----------------------------------------------------------------------
1    Fe-56        (z,z0)          2.4160e-03   100.00%
✓ Low uncertainty level
